### 创建自定义Estimators
这篇tutorial介绍了如何自定义Estimator，特别是，如何保证和我们之前看到的pre-made Estimators具有类似的功能与接口
<br>（我们继续用鸢尾花当例子
<img src = "https://gss2.bdstatic.com/9fo3dSag_xI4khGkpoWK1HF6hhy/baike/c0%3Dbaike80%2C5%2C5%2C80%2C26/sign=6e3294171bd8bc3ed2050e98e3e2cd7b/86d6277f9e2f07082a8b0b2deb24b899a901f214.jpg">

### 1.自定义Estimator与Pre-made Estimator的不同
如👇图所示，pre-made estimator是tf.estimator.Estimator的子类，但是自定义的Estimator则是示例
<img src = "https://www.tensorflow.org/images/custom_estimators/estimator_types.png?hl=zh-cn">

从操作上来看，唯一区别是，自定义Estimator需要自己写实现算法（是不是和没说一样
<br>还是看具体例子好啦, 例如我们想实现鸢尾花的分类问题，那么大概需要建立以下👇这样的模型
<img src = "https://www.tensorflow.org/images/custom_estimators/full_network.png?hl=zh-cn">

### 2.🌰
#### 2.1 Input Function
我们自定义的Estimator的输入函数采用和之前pre-made Estimator相同的实现

In [5]:
import tensorflow as tf
import iris_data
(train_x, train_y), (test_x, test_y) = iris_data.load_data()

In [3]:
def train_input_fn(features, labels, batch_size):
    """An input function for training"""
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle, repeat, and batch the examples.
    dataset = dataset.shuffle(1000).repeat().batch(batch_size)

    # Return the read end of the pipeline.
    return dataset.make_one_shot_iterator().get_next()

#### 2.2 自定义特征列
特征列部分也在<a href = "">之前的tutorial</a>中讲过啦

In [6]:
# Feature columns describe how to use the input.
my_feature_columns = []
for key in train_x.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))

#### 2.3 写模型函数
当定义一个模型函数，我们需要传入从Input function中返回的features以及labels，这些是我们定义的模型直接要用到的数据，mode参数是用来只是我们调用的是何种方法，是train，predict还是evaluate，所以同样的，你需要出这三种模式下所需的计算。
#### 总结来说，我们定义一个模型函数，需要做以下工作：
* 定义模型
* 指定不同模式下的计算
 * Train
 * Evaluate
 * Predict

#### 2.3.1 我们先来看一下定义模型
一个基本的深度神经网络包括以下三部分：
* 一个输入层
* 至少一层隐藏层
* 一个输出层

<BR>
* 定义一个输入层,我们可以直接调用tf.feature_column.input_layer方法来将特征字典和特征列转化为模型输入
    * net = tf.feature_column.input_layer(features, params['feature_columns'])
    <img src = "https://www.tensorflow.org/images/custom_estimators/input_layer.png?hl=zh-cn">
    
<BR>
* 关于定义隐藏层，tensorflow提供的layers接口提供了各种方法，可以自行设置卷积，池化等。但对于鸢尾花数据集，由于该数据集比较简单，我们只需要调用简单的全连接层来定义隐藏层(tf.layers.dense),其中每一层的单元数在params['hidden_layers']中定义
    * for units in params['hidden_units']:
        net = tf.layers.dense(net, units=units, activation=tf.nn.relu)

 关于以上代码一点额外说明：在每次迭代建立隐藏层过程中，当前层以上一层作为输入
     <img src = "https://www.tensorflow.org/images/custom_estimators/add_hidden_layer.png?hl=zh-cn">
 
<BR>
* 关于输出层，这里我们继续调用tf.layers.dense来建立输出层，但注意输出层没有激活函数
   * logits = tf.layers.dense(net, params['n_classes'], activation=None)
   <img src = "https://www.tensorflow.org/images/custom_estimators/add_logits.png?hl=zh-cn">

#### 2.3.2 实现训练，评估和预测模式
我们在定义自己的模型函数时，需要指定这次计算是训练模式，评估模式还是预测模式，那这时候，我们需要在定义自己的模型函数时，出入一个mode参数，来指定本次计算的模式：
<br >def my_model_fn( features, labels, mode, params): 
* 关于mode参数：
  当我们想调用train，evaluate或者predict方法时，我们通过特定的mode参数来唤醒这些方法：
  * ModeKeys.TRAIN ---> train()
  * ModeKeys.EVAL ---> evaluate()
  * ModeKeys.PREDICT ---> predict()
  

#### Calculate the loss
在接下来的训练和评估模式下，我们都需要计算模型的损失，也就是需要被优化的对象；我们可以通过调用tf.losses.sparse_softmax_cross_entropy来计算模型损失：
* loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

#### Train
关于训练模式，当Estimator的train方法被调用时，相应的，模型函数被 mode = ModeKeys.TRAIN 唤醒，在这种情况下，模型函数必须返回一个<a href = "">EstimatorSpec</a>，包含计算得到的损失以及训练方法：
  建立训练操作需要我们定义优化器，我们通常用tf.train.AdagradOptimizer作为默认优化器（tf.train中也提供了许多其他优化器）
  * optimizer = tf.train.AdagradOptimizer(learning_rate=0.1)
  
<br>接下来，我们用刚刚定义的优化器来最小化模型损失，优化器的最小化方法，带有一个global_step参数，TensorFlow用该参数来计算已经训练的次数（来确定训练何时结束）,除此以外，global_step参数对于TensorBoard图形化模型非常必要，示例代码如下：
  * train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())
  
<br>返回的EstimatorSpec必须包括以下部分：
      * loss: 包含损失函数值
      * train_op: 可执行的训练方法
      <br>示例代码如下：
      * return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)


#### Evaluate
类似的，当Estimator的评估方法被调用时，model_fn接收到了mode = ModeKeys.EVAL, 在这种情况下，模型函数需要返回一个包含模型损失以及可选择的矩阵。
<br>(你可能需要查看<a href = "">关于Metric的tutorial</a>)
<br>尽管返回矩阵是可选择的，大多数自定义Estimator都至少会返回一个矩阵，TensorFlow提供了矩阵模块，来计算一些常用矩阵，在当前这个例子中，我们默认返回Accuracy矩阵：
* accuracy = tf.metrics.accuracy(labels=labels,predictions=predicted_classes, name='acc_op')

<BR>
在实际编程中，我们可能不仅创建一个矩阵，所以，在返回结果之前，创建一个字典，将我们创建的每个矩阵附到字典中一并返回，示例代码如下：
* metrics = {'accuracy': accuracy}

最终返回：
* return tf.estimator.EstimatorSpec(mode, loss=loss, eval_metric_ops=metrics)
此外，为了在TensorBoard中，accuracy在Train及Eval模式下都可以可视化，需要加上：
* tf.summary.scalar('accuracy', accuracy[1])


#### Predict
嗯，还是类似的，当Estimator的预测方法被调用，Model_fn将会收到mode = ModeKeys.PREDICT; 
<br>在调用预测模式之前，该模型必须已经训练。训练后的模型存储在model_dir目录下，在初始化Estimator时被重建。
<br>预测模式下，模型范数必须返回相应的预测值, 生成预测值得代码如下：
* predicted_classes = tf.argmax(logits, 1)
* predictions = {
        'class_ids': predicted_classes[:, tf.newaxis],
        'probabilities': tf.nn.softmax(logits),
        'logits': logits,}
<br>predictions字典中包含了我们需要的所有信息：
<img src = "https://www.tensorflow.org/images/custom_estimators/add_predictions.png?hl=zh-cn">
* 我们只需要返回predictions字典即可：
    * return tf.estimator.EstimatorSpec(mode, predictions=predictions)

In [9]:
#以下是刚刚全部内容的整合体
def my_model(features, labels, mode, params):
    """DNN with three hidden layers, and dropout of 0.1 probability."""
    # Create three fully connected layers each layer having a dropout
    # probability of 0.1.
    net = tf.feature_column.input_layer(features, params['feature_columns'])
    for units in params['hidden_units']:
        net = tf.layers.dense(net, units=units, activation=tf.nn.relu)

    # Compute logits (1 per class).
    logits = tf.layers.dense(net, params['n_classes'], activation=None)

    # Compute predictions.
    predicted_classes = tf.argmax(logits, 1)
    if mode == tf.estimator.ModeKeys.PREDICT:
        predictions = {
            'class_ids': predicted_classes[:, tf.newaxis],
            'probabilities': tf.nn.softmax(logits),
            'logits': logits,
        }
        return tf.estimator.EstimatorSpec(mode, predictions=predictions)

    # Compute loss.
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    # Compute evaluation metrics.
    accuracy = tf.metrics.accuracy(labels=labels,
                                   predictions=predicted_classes,
                                   name='acc_op')
    metrics = {'accuracy': accuracy}
    tf.summary.scalar('accuracy', accuracy[1])

    if mode == tf.estimator.ModeKeys.EVAL:
        return tf.estimator.EstimatorSpec(
            mode, loss=loss, eval_metric_ops=metrics)

    # Create training op.
    assert mode == tf.estimator.ModeKeys.TRAIN

    optimizer = tf.train.AdagradOptimizer(learning_rate=0.1)
    train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)

#### 2.4 初始化模型
一些其他的参数，可能需要在建立Estimator中需要用到，这些参数用于设置模型，例如隐藏层的数量，每一层单元的个数，输出层的维度，等等，设置方法与我们之前调用pre-made Estimator时的设置方法类似

In [22]:
classifier = tf.estimator.Estimator(
    model_fn=my_model,
    params={
        'feature_columns': my_feature_columns,
        # Two hidden layers of 10 nodes each.
        'hidden_units': [10, 10],
        # The model must choose between 3 classes.
        'n_classes': 3,
    },
    model_dir = 'model/iris')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'model/iris', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x11ea03160>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


#### 2.5 训练示例
训练，评估，预测等方法，与我们在pre-made Estimator中见到的一样，这里给出简单样例：

In [23]:
# Train the Model.
classifier.train(
    input_fn=lambda:iris_data.train_input_fn(train_x, train_y, 100),
    steps=1000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into model/iris/model.ckpt.
INFO:tensorflow:loss = 1.5508926, step = 1
INFO:tensorflow:global_step/sec: 543.564
INFO:tensorflow:loss = 0.32071263, step = 101 (0.185 sec)
INFO:tensorflow:global_step/sec: 616.435
INFO:tensorflow:loss = 0.06946964, step = 201 (0.162 sec)
INFO:tensorflow:global_step/sec: 557.824
INFO:tensorflow:loss = 0.06918395, step = 301 (0.179 sec)
INFO:tensorflow:global_step/sec: 628.35
INFO:tensorflow:loss = 0.05326658, step = 401 (0.159 sec)
INFO:tensorflow:global_step/sec: 635.684
INFO:tensorflow:loss = 0.09533507, step = 501 (0.157 sec)
INFO:tensorflow:global_step/sec: 542.89
INFO:tensorflow:loss = 0.06499552, step = 601 (0.185 sec)
INFO:tensorflow:global_step/sec: 501.439
INFO:tensorflow:loss = 0.086014576, step = 701 (0.199 sec)
INFO:tensorflow:global_step/sec: 617.628
INFO:tensorflow:loss = 0.11016789, step = 801 (0.161 sec)
INFO:tensorflow:global_step/sec: 682.361
INFO:tensorf

#### 以上就是关于Estimator部分的全部内容啦~